# Лабораторная 1
## Подраздел: Методы Простых итераций и Зейделя 

* Cтудент: Ефимов А.В.
* Группа: М8О-307Б
* Вариант: 7

Загрузка библиотек, матрицы и настройка

In [2]:
import numpy as np
import logging, json
from math import sqrt
from numpy.linalg import norm

# Config
logging.basicConfig(level=logging.DEBUG)
MAX_ITER_COUNT = 1000

with open("task.json", "r") as json_file:
    task = json.load(json_file)

matrix = np.array(task["matrix"])
constants = np.array(task["constants"])
try:
    eps = task["epsilon"]
except KeyError:
    eps = 10e-5
    
print(f"Matrix:\n{matrix}")
print(f"Constants: {constants}")
print(f"Eps: {eps}")

Преобразовать матрицы и константы в эквивалентный вид, при котором сравнительно упрощается
подсчет.

In [3]:
def calc_equivalent(matrix, contants):
    new_matrix = np.zeros_like(matrix)
    new_constants = np.zeros_like(constants)

    diag = matrix.diagonal()

    if not(all(diag != 0)):
            raise ValueError("Diagonal values can not contain 0")

    new_matrix = - matrix / diag[:, np.newaxis]
    for i in range(len(new_matrix)):
        new_matrix[i, i] = 0
    new_constants = constants / diag
    
    return (new_matrix, new_constants)

new_matr, new_consts = calc_equivalent(matrix, constants)
print(f"Equivalent matrix:\n{new_matr}")
print(f"Equivalent constants: {new_consts}")

New matrix:
[[ 0.         -0.27586207 -0.31034483  0.31034483]
 [-0.28        0.         -0.          0.36      ]
 [-0.0625     -0.375       0.          0.125     ]
 [ 0.41176471 -0.23529412  0.11764706  0.        ]]
New constants: [ 6.79310345  9.04       -5.9375     -3.41176471]


Функция проверки на окончание итераций для обоих методов. Структурно она не отличается, 
но меняются коэффиценты делимого и делителя.

In [4]:
def test_exit(prev_res, new_res, coef_divident, coef_divisor, eps):
    difference = new_res - prev_res
    diff_norm = norm(difference)

    if coef_divisor < 1:
        return diff_norm * coef_divident / (1 - coef_divisor) <= eps
    else:
        return diff_norm <= eps

Метод простых итераций для решения системы.

In [8]:
def solve_iter(matrix, constants, eps = 10e-5):
    matrix_norm  = norm(matrix)
    x_values     = constants.copy()
    new_x_values = constants.copy()
    
    logging.debug(f"Original values: {x_values}")
    
    for iter_no in range(MAX_ITER_COUNT):
        logging.debug(f"Iteration no. {iter_no}")
        
        row_sums = np.sum(matrix * x_values, axis=1)
        new_x_values = constants + row_sums
        
        logging.debug(f"Original values: {new_x_values}")

        if test_exit(x_values, new_x_values, matrix_norm, matrix_norm, eps):
            logging.debug("Exit condition achieved")
            break
        else:
            x_values = new_x_values.copy()

    return new_x_values

res_i = solve_iter(new_matr, new_consts, eps)
print("Iterative solution:", res_i)

DEBUG:root:Original values: [ 6.79310345  9.04       -5.9375     -3.41176471]
DEBUG:root:Iteration no. 0
DEBUG:root:Original values: [  5.08315923   5.90969574 -10.17853955  -3.4401927 ]
DEBUG:root:Iteration no. 1
DEBUG:root:Original values: [ 7.25405365  6.37824604 -8.90135744 -3.90669103]
DEBUG:root:Iteration no. 2
DEBUG:root:Original values: [ 6.58365619  5.60245621 -9.271057   -2.97278374]
DEBUG:root:Iteration no. 3
DEBUG:root:Original values: [ 7.20223482  6.12637412 -8.82149756 -3.10978503]
DEBUG:root:Iteration no. 4
DEBUG:root:Original values: [ 6.87566965  5.90385164 -9.0737531  -2.92546164]
DEBUG:root:Iteration no. 5
DEBUG:root:Original values: [ 7.07254517  6.06164631 -8.94685642 -3.03724854]
DEBUG:root:Iteration no. 6
DEBUG:root:Original values: [ 6.95494139  5.96627788 -9.03230751 -2.97838129]
DEBUG:root:Iteration no. 7
DEBUG:root:Original values: [ 7.02603838  6.02039915 -8.9818357  -3.01441981]
DEBUG:root:Iteration no. 8
DEBUG:root:Original values: [ 6.98426034  5.9875181

Iterative solution: [ 6.99999856  5.99999887 -9.00000102 -2.99999925]


In [7]:
def solve_seidel(matrix, constants, eps = 10e-5):
    upper        = np.triu(matrix)
    upper_norm   = norm(upper)
    matrix_norm  = norm(matrix)
    x_values     = constants.copy()
    new_x_values = constants.copy()

    logging.debug(f"Original values: {x_values}")
    
    for iter_no in range(MAX_ITER_COUNT):
        for i in range(len(matrix)):
            logging.debug(f"Iteration no. {iter_no}")

            row_sum = sum(matrix[i] * new_x_values)
            new_x_values[i] = constants[i] + row_sum

            logging.debug(f"Original values: {new_x_values}")

        if test_exit(x_values, new_x_values, upper_norm, matrix_norm, eps):
            logging.debug("Exit condition achieved")
            break
        else:
            x_values = new_x_values.copy()

    return new_x_values

res_s = solve_seidel(new_matr, new_consts, eps)
print("Seidel solution:", res_s)

DEBUG:root:Original values: [ 6.79310345  9.04       -5.9375     -3.41176471]
DEBUG:root:Iteration no. 0
DEBUG:root:Original values: [ 5.08315923  9.04       -5.9375     -3.41176471]
DEBUG:root:Iteration no. 0
DEBUG:root:Original values: [ 5.08315923  6.38848012 -5.9375     -3.41176471]
DEBUG:root:Iteration no. 0
DEBUG:root:Original values: [ 5.08315923  6.38848012 -9.07734809 -3.41176471]
DEBUG:root:Iteration no. 0
DEBUG:root:Original values: [ 5.08315923  6.38848012 -9.07734809 -3.88979424]
DEBUG:root:Iteration no. 1
DEBUG:root:Original values: [ 6.64069461  6.38848012 -9.07734809 -3.88979424]
DEBUG:root:Iteration no. 1
DEBUG:root:Original values: [ 6.64069461  5.78027958 -9.07734809 -3.88979424]
DEBUG:root:Iteration no. 1
DEBUG:root:Original values: [ 6.64069461  5.78027958 -9.00637254 -3.88979424]
DEBUG:root:Iteration no. 1
DEBUG:root:Original values: [ 6.64069461  5.78027958 -9.00637254 -3.09700007]
DEBUG:root:Iteration no. 2
DEBUG:root:Original values: [ 7.03248674  5.78027958 -9

Seidel solution: [ 7.00000041  6.00000076 -9.00000001 -3.00000001]
